# Wakefield example

Simple 1 m drift with a wakefield. 

This verifies that the analytic formula uses is SLAC-PUB-9663 Eq. 8



In [ ]:
import numpy as np
import os

import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams["figure.figsize"] = (8, 4)
%config InlineBackend.figure_format='retina'

In [ ]:
# locate the drift template
from impact import Impact

ifile = "../templates/wakefield/ImpactT.in"
os.path.exists(ifile)

In [ ]:
# gamma for 1 GeV
1e9 / 0.511e6

# Use Impact's built-in Gaussian particle generator

In [ ]:
I = Impact(ifile)
I.header["Np"] = 10000
I.header["Nx"] = 32
I.header["Ny"] = 32
I.header["Nz"] = 32
I.header["Dt"] = 10e-12

In [ ]:
I.lattice

In [ ]:
I.run()

In [ ]:
I.output["stats"].keys()

In [ ]:
PI = I.particles["initial_particles"]
PF = I.particles["final_particles"]
PI, PF

In [ ]:
PI.plot("delta_z", "delta_pz")
PF.plot("delta_z", "delta_pz")

In [ ]:
PF.plot("delta_z", "delta_pz")

In [ ]:
# np.savetxt('/Users/chrisonian/Scratch/impactwake.dat', np.array([PF['z'], PF['pz']]).T)

# Make particles in distgen

In [ ]:
from distgen import Generator

YAML = """
n_particle: 20000
random_type: hammersley
species: electron
start:
  tstart:
    units: sec
    value: 0
  type: time
total_charge:
  units: nC
  value: 1
r_dist:
  sigma_xy:
    units: mm
    value: .001
  type: radial_gaussian
z_dist:
  avg_z:
    units: mm
    value: 0
  sigma_z:
    units: mm
    value: 0.1
  type: gaussian
  

transforms:
  setPz:
    type: set_avg pz
    avg_pz: 
      value: 1
      units: GeV/c
  
"""
G = Generator(YAML)
G.run()
P = G.particles

In [ ]:
I = Impact(ifile, initial_particles=P, verbose=False)
I.run()
PF2 = I.particles["final_particles"]

In [ ]:
PF2.plot("x", "px")
PF2.plot("delta_z", "delta_pz")

# Compare

In [ ]:
for k in ["x", "z", "pz"]:
    plt.hist(PF[k], density=True, bins=100, label="Impact-T generator", alpha=0.5)
    plt.hist(PF2[k], density=True, bins=100, label="Distgen generator", alpha=0.5)
    plt.xlabel(k)
    plt.legend()
    plt.show()

 # Checking the wakefield with SLAC-PUB-9663
 
 Impact-T seems to use Eq. * from SLAC-PUB-9663, Karl Bane (2003). 
 
 https://www.slac.stanford.edu/pubs/slacpubs/9500/slac-pub-9663.pdf
 
 

In [ ]:
# Define alpha function for the s00 calc.


def alpha(g):
    """
    SLAC-PUB-9663 equation (5)

    """
    a1 = 0.4648
    return 1 - a1 * np.sqrt(g) - (1 - 2 * a1) * g


def bane_wake(z, a=0.0116, g=0.0292, L=0.035):
    s00 = g / 8 * (a / (alpha(g / L) * L)) ** 2

    #   'iris_radius': 0.0116,
    #   'gap': 0.0292,
    #   'period': 0.035,

    Z0c_over_pi = 120 * 299792458.0  # Ohm m/s

    WL = Z0c_over_pi / a**2 * np.exp(-np.sqrt(z / s00))

    return WL


def bane_wake2(z, a=0.0116, g=0.0292, L=0.035):
    """
    From SLAC-PUB-11829
    """

    s1 = 0.41 * a**1.8 * g**1.6 / L**2.4

    Z0c_over_pi = 120 * 299792458.0  # Ohm m/s

    WL = Z0c_over_pi / a**2 * np.exp(-np.sqrt(z / s1))

    return WL


plt.xlabel("z (m)")
plt.ylabel("Wake (V/C)")
plt.yscale("log")

dzz = 0.00001
zz = np.arange(0, 0.01, dzz)

plt.yscale("log")
plt.plot(zz, bane_wake(zz), label="SLAC-PUB-9663", color="red")
plt.plot(zz, bane_wake2(zz), label="SLAC-PUB-11829", color="green")
plt.legend()

In [ ]:
# Compare with particles
sigma = 0.0001
Qtot = -1e-9  # C


def density(z, sigma=0.0001):
    return 1 / (np.sqrt(2 * np.pi) * sigma) * np.exp(-0.5 * (z / sigma) ** 2)


dz = sigma / 10
zlist = np.arange(-6 * sigma, 6 * sigma, dz)

# Check normalization
np.sum(density(zlist)) * dz

In [ ]:
def total_bane_wake(z):
    W = bane_wake(zz)
    return np.sum(W * density(zz + z) * dzz)

In [ ]:
plt.xlabel("z")
plt.ylabel(r"$\Delta p_z$ (eV/c)")
plt.scatter(
    PF["delta_z"], PF["pz"] - PF["max_pz"], marker="x", label="Impact-T tracking"
)
plt.plot(
    zlist,
    Qtot * np.array([total_bane_wake(z) for z in zlist]),
    color="blue",
    label="SLAC-PUB-9663 equation (8)",
)
plt.title("Integrated total wake comparison")
plt.legend()

# Comparison with Wakefield file

Many codes will use a wakefield file, with a list of z and single particle wake in V/C

In [ ]:
wfile = "Sz_p5um_10mm_per35mm_cell.sdds"
reffile = os.path.join("../templates/wakefield", wfile)
reffile

In [ ]:
!head -n 8 ../templates/wakefield/Sz_p5um_10mm_per35mm_cell.sdds

In [ ]:
# Load the file
edat = np.loadtxt(reffile, skiprows=7).T
zw = edat[0]
dzw = np.mean(np.diff(zw))
W_from_file = edat[1] / 35e-3  # Convert to per m

plt.ylabel("W (V/C)")
plt.xlabel("z (m)")
plt.yscale("log")
plt.plot(zw, W_from_file, label=wfile)
plt.plot(zw, np.array([bane_wake(z) for z in zw]), label="SLAC-PUB-9663 equation (8)")
plt.scatter(
    zw,
    np.array([bane_wake(z) for z in zw]),
    label="SLAC-PUB-11829 equation (12)",
    color="red",
)
plt.legend()

In [ ]:
def total_wake_from_file(z):
    return np.sum(W_from_file * density(zw + z) * dzw)


total_wake_from_file(0) * 1e-9

In [ ]:
plt.xlabel("z (m)")
plt.ylabel(r"$\Delta p_z$ (eV/c)")
plt.scatter(
    PF["delta_z"], PF["pz"] - PF["max_pz"], marker="x", label="Impact-T tracking"
)
plt.plot(
    zlist,
    Qtot * np.array([total_wake_from_file(z) for z in zlist]),
    color="red",
    label=wfile,
)
plt.plot(
    zlist,
    Qtot * np.array([total_bane_wake(z) for z in zlist]),
    color="blue",
    label="SLAC-PUB-9663 equation (8)",
)
plt.legend()